In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [3]:
train_df = pd.read_csv('Datasets/train.csv',index_col='id')
test_df = pd.read_csv('Datasets/test.csv',index_col='id')

In [ ]:
train_dir = r'P:\Analytics Vidhya\Fashion Mnist\Datasets\train'

In [ ]:
training_data = []
for img in tqdm(os.listdir(train_dir)):
    img_array = cv2.imread(os.path.join(train_dir,img),cv2.IMREAD_GRAYSCALE)
    training_data.append([img_array,train_df.iloc[(int(img.split('.')[0]))-1].label])

In [ ]:
test_dir = r'P:\Analytics Vidhya\Fashion Mnist\Datasets\test'

testing_data = []
for img in tqdm(os.listdir(test_dir)):
    img_array = cv2.imread(os.path.join(test_dir,img),cv2.IMREAD_GRAYSCALE)
    testing_data.append(img_array)

In [ ]:
import random
random.shuffle(training_data)
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

In [ ]:
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [4]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [5]:
IMG_SIZE = 28
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=42)

In [9]:
x_train = x_train/255

x_test = x_test/255

In [ ]:
x_train.shape

In [10]:
from tensorflow.keras.utils import to_categorical

y_cat_train = to_categorical(y_train)
y_cat_test = to_categorical(y_test)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, InputLayer, BatchNormalization, Dropout

In [ ]:

cnn = Sequential()
    
cnn.add(InputLayer(input_shape=(IMG_SIZE,IMG_SIZE,1)))

# Normalization
cnn.add(BatchNormalization())

# Conv + Maxpooling
cnn.add(Conv2D(64, (4, 4), padding='same', activation='relu'))
cnn.add(MaxPool2D(pool_size=(2, 2)))

# Dropout
cnn.add(Dropout(0.1))

# Conv + Maxpooling
cnn.add(Conv2D(64, (4, 4), activation='relu'))
cnn.add(MaxPool2D(pool_size=(2, 2)))

# Dropout
cnn.add(Dropout(0.3))

# Converting 3D feature to 1D feature Vektor
cnn.add(Flatten())

# Fully Connected Layer
cnn.add(Dense(256, activation='relu'))

# Dropout
cnn.add(Dropout(0.5))

# Fully Connected Layer
cnn.add(Dense(64, activation='relu'))

# Normalization
cnn.add(BatchNormalization())

cnn.add(Dense(10, activation='softmax'))
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3),activation='relu', padding='same',input_shape=(IMG_SIZE,IMG_SIZE,1)))
    model.add(BatchNormalization(axis=1))

    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(BatchNormalization(axis=1))

    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))


    model.add(Conv2D(64,(3,3),activation='relu', padding='same',input_shape=(IMG_SIZE,IMG_SIZE,1)))
    model.add(BatchNormalization(axis=1))

    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(BatchNormalization(axis=1))

    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [32, 64, 128, 256]
epochs = [25,35,50,60,80]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
cnn.fit(x_train,y_cat_train,epochs=80,verbose=2 ,validation_data=(x_test, y_cat_test))

In [ ]:
model.fit(x_train,y_cat_train,epochs=25,batch_size=32,verbose=2 ,validation_data=(x_test, y_cat_test))

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict_classes(x_test)

print(classification_report(y_test,predictions))

In [ ]:
x_valid = np.array(testing_data).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
x_valid = x_valid / 255

In [ ]:
pred_val = model.predict_classes(x_valid)

In [ ]:
pred_val

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
output = pd.DataFrame({'id': test_df.index,
                       'label': pred_val})
output.to_csv('submission.csv', index=False)